In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("/Users/skylerwilson/Desktop/PartsMatch/PartsWise/Data/parts_data.csv")
df.head()

,Part Number,Category,Description,Supplier Name,Price,Quantity,Cost Per Unit,Months No Sale,Quantity Ordered YTD,Sales - Jan,...,Order-to-Sales Ratio,Consistency Score,Gross Profit,CLV,Adjusted Demand,Adjusted EOQ,Safety Stock,Reorder Point,Stock Status,Optimal Quantity
0,298670,Essential,"sponsor sticker kit ""fun phras",parts canada/power twins,23.05,0,13.62,1,0,0,...,0.0,80.481094,9.43,83.23,0.161052,1.0,1,1,Understocked,2.0
1,83300402141,Essential,special sprocket,bmw,121.99,0,101.09,1,0,0,...,0.0,93.670458,20.90,169.25,0.101327,1.0,1,1,Understocked,2.0
2,165497,Essential,ctx7l-bs(ytx7l-bs) parts batte,parts canada/power twins,49.95,0,25.97,1,0,1,...,0.0,85.710316,23.98,192.35,8.150989,2.0,1,2,Understocked,3.0
3,46637681427,Essential,l-support windshield,bmw,215.49,0,94.50,1,0,0,...,0.0,100.000000,120.99,919.92,0.074788,1.0,1,1,Understocked,2.0
4,0305-0004,Non-essential,100/90-19 dun d404 fr,parts canada/power twins,160.99,0,104.62,1,0,0,...,0.0,0.000000,56.37,435.28,0.000000,0.0,0,0,At Reorder Point,0.0


,Part Number,Supplier Name,Description,Quantity,Cost Per Unit,Months No Sale,Stock Status,Category,Adjusted Demand,Actual Price,Predicted Price,Price Diff
18501,981073686,ducati,trousers tour c4 woman xl,1,349.30,7,Overstocked,Nearing Obsolete,0.000000,499.99,454.54,7.66
16758,40140061a,ducati,speedometer,0,58.04,7,Optimally Stocked,Nearing Obsolete,0.000000,116.99,106.35,-1.17
5747,t3550380,triumph,"washer, wave",1,3.49,25,Overstocked,Obsolete,0.000000,5.99,3.49,-0.03
8647,t2045744,triumph,"clamp, h/bar, l",0,30.75,18,Optimally Stocked,Obsolete,0.000000,47.99,30.75,0.35
3955,5265832-266,polaris,"support-plate,center,form,blk",12,35.50,29,Overstocked,Obsolete,0.000000,70.99,35.50,0.30
...,...,...,...,...,...,...,...,...,...,...,...,...
4194,1263146-266,polaris,"weld-exh,shield,hdpipe,frt,blk",0,236.24,28,Optimally Stocked,Obsolete,0.000000,314.99,236.24,0.12
4497,7501962447,thibault canada,"agv,x3000,super,helm,",0,318.59,28,Optimally Stocked,Obsolete,0.000000,619.99,318.59,4.70
12645,2862797,polaris,icon pin badge,1,10.39,10,Overstocked,Nearing Obsolete,0.000000,15.99,10.92,0.08
17295,5h0-83980-09-00,argyll moto internationalrsports,front stop switch,0,25.73,7,Optimally Stocked,Nearing Obsolete,0.077364,28.59,25.99,0.96


create the same model as above but only use data that will be available from PartsMatch or something similar:
- Months No Sale --> Time listed on the site
- Quantity --> total quantity matching a particular part number
- Demand --> total sales in a given time period for that part
- Competitive pricing --> Avg or mean price of competitors
- Target --> Price of that part number

Ill have to simplify it for now because I only have dads dealership data but this is similar to what ill build for the ML-algorithmic pricing for PartsMatch

In [ ]:
cols = ['Sales - Jan', 'Sales - Feb', 'Sales - Mar', 'Sales - Apr',
       'Sales - May', 'Sales - Jun', 'Sales - Jul', 'Sales - Aug',
       'Sales - Sep', 'Sales - Oct', 'Sales - Nov', 'Sales - Dec',
       'Months No Sale', 'Quantity', 'Cost Per Unit', 'Gross Profit',
       'Margin %']
PartsMatch = df[cols]

In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import RobustScaler, PowerTransformer

# Assuming df is your dataframe and is already loaded
X = PartsMatch.select_dtypes(exclude='object')
y = df['Price']

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=42)

# Define the model
model = XGBRegressor()

# Define the transformer
transformer = PowerTransformer(method='yeo-johnson')

# Define the scaler
scaler = RobustScaler()

# Create a pipeline for the features
feature_pipeline = Pipeline([
    ('transformer', transformer),
    ('scaler', scaler)
])

# Combine the feature pipeline and the model into a single pipeline
# and apply a power transformation to the target variable
final_pipeline = TransformedTargetRegressor(
    regressor=Pipeline([
        ('feature_pipeline', feature_pipeline),
        ('model', model)
    ]),
    transformer=PowerTransformer(method='yeo-johnson')
)

# Define the parameter grid
param_dist = {
    'regressor__model__n_estimators': [250, 275, 300],
    'regressor__model__learning_rate': [0.01, 0.05, 0.1, 0.25],
    'regressor__model__reg_alpha': [0.01, 0.05, 0.1],
    'regressor__model__reg_lambda': [1, 1.25, 1.5],
    'regressor__model__scale_pos_weight': [1, 2, 3],
    'regressor__model__base_score': [0.5],
    'regressor__model__objective': ['reg:squarederror'],
    'regressor__model__booster': ['gbtree', 'gblinear', 'dart'],
    'regressor__model__eval_metric': ['rmse'],
    'regressor__model__random_state': [42],
}

# Set up cross-validation
cv = KFold(n_splits=3, random_state=42, shuffle=True)

# Set up RandomizedSearchCV
search = RandomizedSearchCV(estimator=final_pipeline, param_distributions=param_dist, n_iter=10, cv=cv, random_state=42, n_jobs=-1)

# Fit the model
search.fit(X_train, y_train)

# Print the best parameters
print("Best Parameters:")
print(search.best_params_)

# Make predictions on the validation set
predictions = search.predict(X_val)

# Calculate performance metrics
mse = mean_squared_error(y_val, predictions)
r2 = r2_score(y_val, predictions)
mae = mean_absolute_error(y_val, predictions)

print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')
print(f'Mean Absolute Error: {mae}')

Best Parameters:
{'regressor__model__scale_pos_weight': 3, 'regressor__model__reg_lambda': 1, 'regressor__model__reg_alpha': 0.05, 'regressor__model__random_state': 42, 'regressor__model__objective': 'reg:squarederror', 'regressor__model__n_estimators': 275, 'regressor__model__learning_rate': 0.05, 'regressor__model__eval_metric': 'rmse', 'regressor__model__booster': 'dart', 'regressor__model__base_score': 0.5}
Mean Squared Error: 38237.24636799818
R^2 Score: 0.8212963409868623
Mean Absolute Error: 7.13861954144267


In [ ]:
additional_info_2 = df.loc[X_val.index, ['Part Number', 'Supplier Name', 'Description', 'Quantity', 
                                       'Cost Per Unit', 'Months No Sale',
                                       'Understocked', 'Overstocked', 'Essential', 'Non-essential',
                                       'Nearing Obsolete', 'Obsolete', 'Adjusted Demand']]
        
# Now 'predictions' contains the prices in the original scale, 
# and you can compare them directly to 'y_val'
comparison_df_2 = pd.DataFrame({'Actual Price': y_val, 'Predicted Price': np.round(predictions, 2)})

# Merge the dataframes
comparison_df_2 = additional_info.merge(comparison_df_2, left_index=True, right_index=True)

comparison_df_2['Price Diff'] = np.round(comparison_df_2['Actual Price'] - comparison_df_2['Predicted Price'], 2)

comparison_df_2['Predicted Price'] = comparison_df_2.apply(
    lambda row: adjust_price(row['Actual Price'], row['Months No Sale'], row['Cost Per Unit']),
    axis=1
)

comparison_df_2['Predicted Price'] = round(comparison_df_2['Predicted Price'], 2)

comparison_df_2

,Part Number,Supplier Name,Description,Quantity,Cost Per Unit,Months No Sale,Understocked,Overstocked,Essential,Non-essential,Nearing Obsolete,Obsolete,Adjusted Demand,Actual Price,Predicted Price,Price Diff
18501,981073686,ducati,trousers tour c4 woman xl,1,349.30,7,0,1,0,0,1,0,0.000000,499.99,454.54,6.52
49768,t2046501,triumph,clutch lever,0,2.91,0,0,0,0,1,0,0,0.000000,67.99,67.99,-3.19
16758,40140061a,ducati,speedometer,0,58.04,7,0,0,0,0,1,0,0.000000,116.99,106.35,-0.99
49319,t16-500,moto international,tube,0,8.26,0,0,0,0,1,0,0,0.000000,34.95,34.95,-2.41
5747,t3550380,triumph,"washer, wave",1,3.49,25,0,1,0,0,0,1,0.000000,5.99,3.49,-0.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12645,2862797,polaris,icon pin badge,1,10.39,10,0,1,0,0,1,0,0.000000,15.99,10.92,0.04
2780,34317707371,bmw,tandem clamp,0,2.72,1,1,0,1,0,0,0,0.020320,4.49,4.49,-0.01
17295,5h0-83980-09-00,argyll moto internationalrsports,front stop switch,0,25.73,7,0,0,0,0,1,0,0.077364,28.59,25.99,0.78
14856,0340-0813,parts canada/power twins,160/60r15 67h diablo rosso rear,0,256.72,7,0,0,0,0,1,0,0.033954,394.99,359.08,-3.64
